In [ ]:
import cv2
import time

# Create a VideoCapture object to read from the video file
cap = cv2.VideoCapture('video_cap.mp4')

# coordinates of the border region (left, top, right, bottom)
border_coordinates = (400, 400, 800, 800)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
else:
    # Read the first frame from the video
    ret, prev_frame = cap.read()
    frame_count = 0

    # Check if the frame was read successfully
    if not ret:
        print("Error: Couldn't read the first frame.")
    else:
        while True:
            # Read the next frame from the video
            ret, current_frame = cap.read()
            real_current_frame = current_frame

            # If the frame was not read successfully, break the loop
            if not ret:
                break

            # Crop the frames to the specified border region
            prev_frame_cropped = prev_frame[border_coordinates[1]:border_coordinates[3], border_coordinates[0]:border_coordinates[2]]
            current_frame_cropped = current_frame[border_coordinates[1]:border_coordinates[3], border_coordinates[0]:border_coordinates[2]]

            # Compute the absolute difference between the cropped frames
            difference = cv2.absdiff(prev_frame_cropped, current_frame_cropped)
            threshold = 30

            # Convert the difference image to grayscale
            gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)

            print(cv2.countNonZero(gray_difference))
            
            # If a significant difference is detected, save the frame
            if cv2.countNonZero(gray_difference) >= 100000:

                # Apply a threshold to highlight the differences
                _, thresholded_diff = cv2.threshold(gray_difference, 30, 255, cv2.THRESH_BINARY)
                # Find contours in the thresholded difference image
                contours, _ = cv2.findContours(thresholded_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Draw rectangles around the detected changes on the original image
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    # Adjust the coordinates to match the original frame
                    x += border_coordinates[0]
                    y += border_coordinates[1]
                    cv2.rectangle(prev_frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

                # Display the original image with changes highlighted
                imS = cv2.resize(prev_frame, (9602, 740))
                cv2.imshow('Difference Image', prev_frame_cropped)

                filename = f'diff_framenew_{frame_count}.jpg'
                cv2.imwrite(filename, prev_frame)

            frame_count += 1

            # Update the previous frame for the next iteration
            prev_frame = real_current_frame

            # If the user presses 'q', exit the loop
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        # Release the VideoCapture and close OpenCV windows
        cap.release()
        cv2.destroyAllWindows()
